In [1]:
#Basic Imports
import pandas as pd
import os

#URL Image Download
import requests

#Google Cloud
from google.cloud import storage

In [ ]:
# Save the excel file as a DataFrame to 'data'
file_path = 'directory_consumer_grade_images.xlsx'
data = pd.read_excel(file_path)
data.tail()

In [ ]:
#Checking the number of images per unique NDC11 ID which is the identifier for each pill.
data['NDC11'].value_counts()

In [ ]:
# Only taking the top five pills with the most images.
#data = data[(data['NDC11'] == 49884041301)] # Only 10 images for testing!

data = data[(data['NDC11'] == 555901258) | (data['NDC11'] == 555903270) | (data['NDC11'] == 63459070160) | (data['NDC11'] == 52544029128) | (data['NDC11'] == 39022310)]
print(f'The total number of images is: {len(data)}')

In [ ]:
# Setting up all the urls from which to download the images as a pandas Series.
pre_url = 'https://data.lhncbc.nlm.nih.gov/public/Pills/'
post_url = data['Image'] 
full_url = pre_url + post_url

In [ ]:
LOCAL_DATA_PATH = os.path.join(os.path.expanduser('~'), "Downloads", "images")
key_file_path = os.path.join(os.path.expanduser('~'), ".pill-pic", "pill-pic-a4bcd429b85c.json") # Add the gcloud service acc JSON key to a .pill-pic folder.
BUCKET_NAME = "pill_pic_image_set"
counter = 1

client = storage.Client.from_service_account_json(key_file_path)
bucket = client.get_bucket(BUCKET_NAME)

for index, url in full_url.items():
    
    file_extension = (os.path.splitext(url)[1]).lower()  # Gets the file extension from the url
    image_path = os.path.join(LOCAL_DATA_PATH, f"{index}{file_extension}") # Complete file path including image
    
    # Downloads the image from url and saves it locally.
    response = requests.get(url)
    with open(image_path, 'wb') as file:
        file.write(response.content)
    print(f"{index}{file_extension} saved to {LOCAL_DATA_PATH}.")
    
    # Uploads the image to Google Cloud.
    blob = bucket.blob(f"{index}{file_extension}")
    blob.upload_from_filename(image_path)
    print(f"{index}{file_extension} uploaded to {BUCKET_NAME} on Google Cloud. \n ✅ {counter}/{len(data)}")
    counter += 1
